In [0]:
!pip install pydicom


In [0]:
#import libraries

import pydicom
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
import shutil
import scipy.ndimage
from skimage.segmentation import clear_border
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from PIL import Image
import scipy.misc
from google.colab.patches import cv2_imshow

from scipy import ndimage as ndi
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data

#for saving data to google drive
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#drive data directory
data_dir = 'gdrive/My Drive/LUNG_DATA/' 
patients = os.listdir(data_dir)
patients.remove('_PROCESSED')

# Labeled scans: 318 
# Unlabeled scans: 104
df = pd.DataFrame(columns=['image_slice', 'cancer']) #holistic images dataframe
df2 = pd.DataFrame(columns=['image_slice', 'cancer']) #patch images dataframe

len(patients)


317

In [0]:
def load_scan(patient):
    slices_path = data_dir + patient + '/slices'
    slices = [pydicom.read_file(slices_path + '/' + s) for s in os.listdir(slices_path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices


In [0]:
def load_contour_data(patient):
  structures_path = data_dir + patient + '/structures'
  structures = [pydicom.read_file(structures_path + '/' + s) for s in os.listdir(structures_path)]

  ctrs = structures[0].ROIContourSequence
  if len(ctrs) != 0:
    contours = [np.array(i.ContourData) for i in ctrs[0].ContourSequence]
    contours = np.array(contours)
  else:
    contours = []
    
  return contours
 
 

In [0]:

#some scans were incorrectly callibrated; the values are the translation amounts along the y axis

problematic_pairs = {'LUNG1-034':-165, 'LUNG1-040':-180, 'LUNG1-044':-165, 'LUNG1-083':210, 'LUNG1-084':195, 'LUNG1-137':175, 'LUNG1-094':180,
                    'LUNG1-096':190, 'LUNG1-110':170, 'LUNG1-143':180, 'LUNG1-146':195, 'LUNG1-144':180,
                    'LUNG1-166':180, 'LUNG1-164':170, 'LUNG1-192':210, 'LUNG1-191':-205, 'LUNG1-208':-180,
                    'LUNG1-222':-180,'LUNG1-212':-170,'LUNG1-214':-180,'LUNG1-227':-180}


#extracts patch from selected slice, image and contour for images labeled as cancer
def get_contour_patch(current_slice, image, contour, patch_dim):
  
  #rearrange the pixel spacing of problematic pairs
  x_spacing, y_spacing = float(current_slice.PixelSpacing[0]), float(current_slice.PixelSpacing[1])
  origin_x, origin_y, _ = current_slice.ImagePositionPatient
  
  if current_slice.PatientID in problematic_pairs:
    origin_y += problematic_pairs[current_slice.PatientID]
  
  contour = np.delete(contour, np.arange(2, contour.size, 3))
  
  contour[0::2] = np.ceil((contour[0::2] - origin_x) / x_spacing)
  contour[1::2] = np.ceil((contour[1::2] - origin_y) / y_spacing)

  
  contour = contour.round().reshape((-1,1,2)).astype(int)
    
  M = cv.moments(contour)
  
  #area of patch region is not zero
  if M["m00"] != 0:
    
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

  else: #revert to bounding rect mode so to avoid division by zero. This method will only approximate the centroid using a bounding box
    x,y,w,h = cv.boundingRect(contour)
    cX = int(x + w/2)
    cY = int(y + h/2)
    
  patch_dim = patch_dim // 2
  image = image[cY-patch_dim:cY+patch_dim, cX-patch_dim:cX+patch_dim]
  
  return image

  
#gets random patch from images labeled as non-cancer
def get_random_patch(image, mask, patch_dim):
  
  #we need only one half of the mask, does not matter which (choose the left one)
  mask = mask[0:512, 0:256]
  M = cv.moments(mask)
  
  # calculate x,y coordinate of centroid
  cX = int(M["m10"] / M["m00"])
  cY = int(M["m01"] / M["m00"])
  
  #crop half the patch size in each dimension
  patch_dim = patch_dim // 2
  
  image = image[cY-patch_dim:cY+patch_dim, cX-patch_dim:cX+patch_dim]
  
  return image

In [0]:

root_path = './gdrive/My Drive/LUNG_DATA/_PROCESSED/'

#main loop to process all patients
for index, patient in enumerate(patients):
  print('Processing scan {} ({}) out of {}'.format(index+1+200, patient, len(patients)))
  
  #loading scans, image pixels and contours
  scan = load_scan(patient)
  pixels = get_pixels_hu(scan)
  contours = load_contour_data(patient)
  
  #segmenting the lungs; this takes a list of slices and returns a list of segmented images
  segmented_lungs_fill = segment_lung_mask(pixels, True)

  slices_cancer = set()
  
  is_cancer = 1
  folder_name = 'cancer_patch/'

  #for each contour, tests if it lies on each slice of each scan
  for cnt_num, contour in enumerate(contours):

    for i, current_slice in enumerate(scan):

      #contour is on current slice
      if current_slice.SliceLocation == contour[2]:

        #add it to cancer list
        slices_cancer.add(i)

        original_image = pixels[i].copy()
        
        #get the patch from the contour
        patch_cancer = get_contour_patch(current_slice, original_image.copy(), contour, patch_dim=64)
        
        #normalization steps
        patch_cancer = normalize(patch_cancer)
        patch_cancer = cv.normalize(patch_cancer, None, alpha = 0, beta = 255, norm_type = cv.NORM_MINMAX, dtype = cv.CV_32F) #rescale between 0,255 
        patch_cancer.astype(np.uint8)  
        
        #saving the image and corresponding labels
        image_name = patient + '-SLICE{0:0=3d}-CONTOUR{1}.jpg'.format(len(scan)-i, cnt_num)
        im_path = root_path + folder_name + image_name
        df2.loc[len(df2)] = [image_name, is_cancer]
        mpimg.imsave(im_path, patch_cancer, cmap=plt.cm.gray, dpi=300)


  is_cancer = 0
  folder_name = 'non_cancer_patch/'
  
  for i, current_slice in enumerate(scan):

    #anything that is not a cancer slice
    if i not in slices_cancer:
      
      #get the pixels and mask of current slice
      original_image = pixels[i].copy()
      mask = segmented_lungs_fill[i].copy().astype(np.int16)
      mask_dilated = cv.dilate(mask, circ_kernel, iterations=1)
      

      #consider the image only if there is more than 32000 black pixels
      if cv.countNonZero(mask_dilated) > 32000:

        #gets random patch from lung region
        patch_non_cancer = get_random_patch(original_image.copy(), mask_dilated, patch_dim=64)
        
        #normalization steps
        patch_non_cancer = normalize(patch_non_cancer)
        patch_non_cancer = cv.normalize(patch_non_cancer, None, alpha = 0, beta = 255, norm_type = cv.NORM_MINMAX, dtype = cv.CV_32F) #rescale between 0,255 
        patch_non_cancer.astype(np.uint8)
        
        #saving image
        image_name = patient + '-SLICE{0:0=3d}.jpg'.format(len(scan)-i)
        im_path = root_path + folder_name + image_name
        df2.loc[len(df2)] = [image_name, is_cancer]
        mpimg.imsave(im_path, patch_non_cancer, cmap=plt.cm.gray, dpi=300)
        


#save dataframe to csv
df2.to_csv(path_or_buf='./gdrive/My Drive/LUNG_DATA/_PROCESSED/labels_patch.csv', index=False)




In [0]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
#credits to Guido Zuidhof: https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

#credits to Guido Zuidhof: https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [0]:


#credits to Guido Zuidhof: https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True) #finds unique labels
    #also return the number of times each unique item appears in ar (return_counts)
    
    
    #everything apart from bg
    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0: #there is at least one component
        return vals[np.argmax(counts)] #return the component with the most counts
    else:
        return None #if there is no component found

      
      
#credits to Guido Zuidhof: https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    
    #finds all connected components and gives each a unique label
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2   
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
#             fig,axes = plt.subplots(1,4,figsize=(24,16))
          
#             axes[0].imshow(np.array(image[i]), cmap='Greys_r')
            
            axial_slice[420:] = 2 #crop problematic big structures in the bottom of slice
            axial_slice = axial_slice - 1
            
#             axes[1].imshow(axial_slice, cmap='Greys_r')

            #finds connected components in slice
            labeling = measure.label(axial_slice)
            
            #finds the largest component
            l_max = largest_label_volume(labeling, bg=0)
            

            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1 #set everything apart from the largest component to 1

#             axes[2].imshow(binary_image[i], cmap='Greys_r')
#             axes[3].imshow(1-binary_image[i], cmap='Greys_r')
  
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs (largest component) are now 1
    
    # Remove other air pockets inside body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0

    
    return binary_image

In [0]:
#circular kernel used in dilation to get smoother result
circ_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))


#this loop processes the holistic images
for i, patient in enumerate(patients[:1]):

  print('Processing scan {} ({}) out of {}'.format(i+1, patient, len(patients)))

  scan = load_scan(patient)
  scan_pixels = get_pixels_hu(scan)

  contours = load_contour_data(patient)
  contour_count = 0
  is_cancer = 0


  if contours != []:

    print('Segmenting lungs...')
    segmented_lungs_fill = segment_lung_mask(scan_pixels, True)

    for i, current_slice in enumerate(scan):
      print('Processing slice {}'.format(i))

      image_name = patient + '-SLICE{0:0=3d}.jpg'.format(len(scan)-i)
      im_path = './gdrive/My Drive/LUNG_DATA/_PROCESSED/'

      mask = segmented_lungs_fill[i].copy().astype(np.int16)
      mask_dilated = cv.dilate(mask, circ_kernel, iterations=1)
      final_result = scan_pixels[i].copy()

      informative_pixels = cv.countNonZero(mask_dilated)

      #if there is a contour in particular slice, then label the image as cancer
      if(contour_count < len(contours) and current_slice.SliceLocation == contours[contour_count][2]):
        contour_count += 1
        is_cancer = 1
        folder_name = 'cancer/'
      else:
        is_cancer = 0
        folder_name = 'non_cancer/'

      print('Informative pixels:',informative_pixels)

      if informative_pixels > 10000:

        final_result[mask_dilated==0] = -1024


        final_result = normalize(final_result) #normalize intensity of image
        final_result = cv.normalize(final_result, None, alpha = 0, beta = 255, norm_type = cv.NORM_MINMAX, dtype = cv.CV_32F) #rescale between 0,255 
        final_result.astype(np.uint8)
          
        im_path = im_path + folder_name + image_name
        df.loc[len(df)] = [image_name, is_cancer]
        mpimg.imsave(im_path, final_result, cmap=plt.cm.gray, dpi=300)


      
df.to_csv(path_or_buf='./gdrive/My Drive/LUNG_DATA/_PROCESSED/labels_new.csv', index=False)
  